# Базовый пример с логистической регрессией

In [1]:
import os

# Used to read the Parquet data
import pyarrow.parquet as parquet
# Used to train the baseline model
from sklearn.linear_model import LogisticRegression


input_path = '../input'
print(os.listdir(input_path))

['train', 'test']


Посмотри на все данные, что у нас есть (аккуратно, очень большой список!)

In [2]:
%%sh
ls -lhR ../input | awk '
/:$/&&f{s=$0;f=0}
/:$/&&!f{sub(/:$/,"");s=$0;f=1;next}
NF&&f{ print s"/"$0 }'

../input/total 8.0K
../input/drwxr-xr-x  2 root root 4.0K Feb  8 18:12 test
../input/drwxr-xr-x 50 root root 4.0K Feb  8 18:13 train
../input/test/total 126M
../input/test/-rw-r--r-- 1 root root    0 Feb  8 18:12 _SUCCESS
../input/test/-rw-r--r-- 1 root root  20K Feb  8 18:12 _common_metadata
../input/test/-rw-r--r-- 1 root root 199K Feb  8 18:12 _metadata
../input/test/-rw-r--r-- 1 root root  18M Feb  8 18:12 part-00000-6d949390-48b0-4104-a477-39e306b726c5-c000.gz.parquet
../input/test/-rw-r--r-- 1 root root  18M Feb  8 18:12 part-00001-6d949390-48b0-4104-a477-39e306b726c5-c000.gz.parquet
../input/test/-rw-r--r-- 1 root root  18M Feb  8 18:12 part-00002-6d949390-48b0-4104-a477-39e306b726c5-c000.gz.parquet
../input/test/-rw-r--r-- 1 root root  19M Feb  8 18:12 part-00003-6d949390-48b0-4104-a477-39e306b726c5-c000.gz.parquet
../input/test/-rw-r--r-- 1 root root  18M Feb  8 18:12 part-00004-6d949390-48b0-4104-a477-39e306b726c5-c000.gz.parquet
../input/test/-rw-r--r-- 1 root root  18M Feb 

Прочтиаем данные за один день, чтобы обучить нашу логистическую регрессию

In [3]:
data = parquet.read_table(input_path + '/train/date=2018-02-07').to_pandas()
data.head(10)

,instanceId_userId,instanceId_objectType,instanceId_objectId,audit_pos,audit_clientType,audit_timestamp,audit_timePassed,audit_experiment,audit_resourceType,metadata_ownerId,metadata_ownerType,metadata_createdAt,metadata_authorId,metadata_applicationId,metadata_numCompanions,metadata_numPhotos,metadata_numPolls,metadata_numSymbols,metadata_numTokens,metadata_numVideos,metadata_platform,metadata_totalVideoLength,metadata_options,relationsMask,userOwnerCounters_USER_FEED_REMOVE,userOwnerCounters_USER_PROFILE_VIEW,userOwnerCounters_VOTE_POLL,userOwnerCounters_USER_SEND_MESSAGE,userOwnerCounters_USER_DELETE_MESSAGE,userOwnerCounters_USER_INTERNAL_LIKE,userOwnerCounters_USER_INTERNAL_UNLIKE,userOwnerCounters_USER_STATUS_COMMENT_CREATE,userOwnerCounters_PHOTO_COMMENT_CREATE,userOwnerCounters_MOVIE_COMMENT_CREATE,userOwnerCounters_USER_PHOTO_ALBUM_COMMENT_CREATE,userOwnerCounters_COMMENT_INTERNAL_LIKE,userOwnerCounters_USER_FORUM_MESSAGE_CREATE,userOwnerCounters_PHOTO_MARK_CREATE,userOwnerCounters_PHOTO_VIEW,userOwnerCounters_PHOTO_PIN_BATCH_CREATE,...,auditweights_matrix,auditweights_notOriginalPhoto,auditweights_numDislikes,auditweights_numLikes,auditweights_numShows,auditweights_onlineVideo,auditweights_partAge,auditweights_partCtr,auditweights_partSvd,auditweights_processedVideo,auditweights_relationMasks,auditweights_source_LIVE_TOP,auditweights_source_MOVIE_TOP,auditweights_svd_prelaunch,auditweights_svd_spark,auditweights_userAge,auditweights_userOwner_CREATE_COMMENT,auditweights_userOwner_CREATE_IMAGE,auditweights_userOwner_CREATE_LIKE,auditweights_userOwner_IMAGE,auditweights_userOwner_MOVIE_COMMENT_CREATE,auditweights_userOwner_PHOTO_COMMENT_CREATE,auditweights_userOwner_PHOTO_MARK_CREATE,auditweights_userOwner_PHOTO_VIEW,auditweights_userOwner_TEXT,auditweights_userOwner_UNKNOWN,auditweights_userOwner_USER_DELETE_MESSAGE,auditweights_userOwner_USER_FEED_REMOVE,auditweights_userOwner_USER_FORUM_MESSAGE_CREATE,auditweights_userOwner_USER_INTERNAL_LIKE,auditweights_userOwner_USER_INTERNAL_UNLIKE,auditweights_userOwner_USER_PRESENT_SEND,auditweights_userOwner_USER_PROFILE_VIEW,auditweights_userOwner_USER_SEND_MESSAGE,auditweights_userOwner_USER_STATUS_COMMENT_CREATE,auditweights_userOwner_VIDEO,auditweights_userOwner_VOTE_POLL,auditweights_x_ActorsRelations,auditweights_likersSvd_spark_hyper,auditweights_source_PROMO
0,189,Post,19851617,10,API,1517999095791,784813,XPRM-5386_q4x2,8,35736,GROUP_OPEN_OFFICIAL,1517994121000,680728,0,0,10,0,40,4,0,OTHER,0,"[HAS_TEXT, HAS_PHOTOS, IS_PART_OF_TOPIC]",NaN,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.004123,NaN,26.0,37.0,11780.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.626212,NaN,41.0,0.937027,NaN,0.987872,0.999999,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,384,Post,11221208,7,WEB,1517961558224,2972959,XPRM-5386_q2x2,8,25338,GROUP_OPEN_OFFICIAL,1517824512000,713108,0,0,1,0,71,12,0,OTHER,0,"[HAS_TEXT, HAS_PHOTOS, IS_PART_OF_TOPIC]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.011834,NaN,10.0,1.0,1322.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.670187,NaN,110.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,384,Post,22510814,11,WEB,1517961558224,2972959,XPRM-5386_q2x2,8,14962,GROUP_OPEN_OFFICIAL,1517582951000,928003,0,0,1,1,0,0,0,WEB,0,"[HAS_PHOTOS, HAS_POLLS, IS_PART_OF_TOPIC]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.006693,NaN,566.0,5366.0,140852.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.706036,NaN,110.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,384,Post,12870307,17,WEB,1517961558224,2972959,XPRM-5386_q2x2,8,46044,GROUP_OPEN_OFFICIAL,1517864769000,672604,0,0,9,0,158,21,0,OTHER,0,"[HAS_TEXT, HAS_PHOTOS, HAS_URLS, IS_PART_OF_TO...",NaN,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.030000,NaN,122.0,376.0,20317.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

Соберем наш таргет...

In [4]:
# Construct the label (liked objects)
y = data['feedback'].apply(lambda x: 1.0 if("Liked" in x) else 0.0).values

...и наши фичи

In [5]:
# Extract the most interesting features
X = data[[
        'auditweights_svd_prelaunch', 
        'auditweights_ctr_high', 
        'auditweights_ctr_gender', 
        'auditweights_friendLikes']].fillna(0.0).values

Обучим нашу регрессию

In [6]:
model = LogisticRegression(random_state=23, solver='lbfgs').fit(X, y)
model.coef_

array([[ 4.6212144 ,  2.69850023, -0.19539506,  0.12107017]])

Модель готова, теперь загрузим тестовые данные...

In [7]:
# Read the test data
test = parquet.read_table(input_path + '/test').to_pandas()
test.head(10)

,instanceId_userId,instanceId_objectType,instanceId_objectId,audit_pos,audit_clientType,audit_timestamp,audit_timePassed,audit_experiment,audit_resourceType,metadata_ownerId,metadata_ownerType,metadata_createdAt,metadata_authorId,metadata_applicationId,metadata_numCompanions,metadata_numPhotos,metadata_numPolls,metadata_numSymbols,metadata_numTokens,metadata_numVideos,metadata_platform,metadata_totalVideoLength,metadata_options,relationsMask,userOwnerCounters_USER_FEED_REMOVE,userOwnerCounters_USER_PROFILE_VIEW,userOwnerCounters_VOTE_POLL,userOwnerCounters_USER_SEND_MESSAGE,userOwnerCounters_USER_DELETE_MESSAGE,userOwnerCounters_USER_INTERNAL_LIKE,userOwnerCounters_USER_INTERNAL_UNLIKE,userOwnerCounters_USER_STATUS_COMMENT_CREATE,userOwnerCounters_PHOTO_COMMENT_CREATE,userOwnerCounters_MOVIE_COMMENT_CREATE,userOwnerCounters_USER_PHOTO_ALBUM_COMMENT_CREATE,userOwnerCounters_COMMENT_INTERNAL_LIKE,userOwnerCounters_USER_FORUM_MESSAGE_CREATE,userOwnerCounters_PHOTO_MARK_CREATE,userOwnerCounters_PHOTO_VIEW,userOwnerCounters_PHOTO_PIN_BATCH_CREATE,...,auditweights_matrix,auditweights_notOriginalPhoto,auditweights_numDislikes,auditweights_numLikes,auditweights_numShows,auditweights_onlineVideo,auditweights_partAge,auditweights_partCtr,auditweights_partSvd,auditweights_processedVideo,auditweights_relationMasks,auditweights_source_LIVE_TOP,auditweights_source_MOVIE_TOP,auditweights_svd_prelaunch,auditweights_svd_spark,auditweights_userAge,auditweights_userOwner_CREATE_COMMENT,auditweights_userOwner_CREATE_IMAGE,auditweights_userOwner_CREATE_LIKE,auditweights_userOwner_IMAGE,auditweights_userOwner_MOVIE_COMMENT_CREATE,auditweights_userOwner_PHOTO_COMMENT_CREATE,auditweights_userOwner_PHOTO_MARK_CREATE,auditweights_userOwner_PHOTO_VIEW,auditweights_userOwner_TEXT,auditweights_userOwner_UNKNOWN,auditweights_userOwner_USER_DELETE_MESSAGE,auditweights_userOwner_USER_FEED_REMOVE,auditweights_userOwner_USER_FORUM_MESSAGE_CREATE,auditweights_userOwner_USER_INTERNAL_LIKE,auditweights_userOwner_USER_INTERNAL_UNLIKE,auditweights_userOwner_USER_PRESENT_SEND,auditweights_userOwner_USER_PROFILE_VIEW,auditweights_userOwner_USER_SEND_MESSAGE,auditweights_userOwner_USER_STATUS_COMMENT_CREATE,auditweights_userOwner_VIDEO,auditweights_userOwner_VOTE_POLL,auditweights_x_ActorsRelations,auditweights_likersSvd_spark_hyper,auditweights_source_PROMO
0,681,Post,35119221,7,API,1521813396697,360522,XPRM-5386_G10,8,14952,GROUP_OPEN_OFFICIAL,1521738031000,613509,0,0,11,0,2257,313,0,WEB,0,"[HAS_TEXT, HAS_PHOTOS, IS_PART_OF_TOPIC]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.003061,NaN,32.0,400.0,24083.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.559876,57.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.927496,NaN
1,681,Post,35151632,32,API,1521904815253,3314178,XPRM-5386_G10,8,9182,GROUP_OPEN_OFFICIAL,1521846310000,1026667,0,0,9,0,4,0,3,OTHER,0,"[HAS_TEXT, HAS_PHOTOS, HAS_VIDEOS, IS_PART_OF_...",NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.001125,NaN,203.0,2852.0,103615.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.228634,NaN,57.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.858149,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,681,Post,20314212,9,API,1522394611865,803588,XPRM-5386_G10,8,16395,GROUP_OPEN_OFFICIAL,1522382496000,1062699,0,0,0,1,108,12,7,OTHER,0,"[HAS_TEXT, HAS_VIDEOS, HAS_POLLS, IS_PART_OF_T...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000060,NaN,226.0,1299.0,22456.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.779892,NaN,57.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,512.0,NaN,NaN
3,1428,Post,27968107,3,API,1522205864350,27994833,XPRM-5386_G4,8,15993,GROUP_OPEN_OFFICIAL,1522165211000,865609,0,0,1,0,245,34,0,OTHER,0,"[HAS_TEXT, HAS_PHOTOS, HAS_URLS, IS_PART_OF_TO...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.004642,NaN,20.0,147.0,6663.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

...и поскорим их.

In [8]:
test["predictions"] = model.predict_proba(
    test[[
        'auditweights_svd_prelaunch', 
        'auditweights_ctr_high', 
        'auditweights_ctr_gender', 
        'auditweights_friendLikes'
    ]].fillna(0.0).values
)[:, 1]

Теперь отсортируем наши предсказания по убыванию и сгруппируем их в списки для каждого пользователя с помощью `groupby().apply(list)`

In [9]:
result_columns = ['instanceId_userId', 'instanceId_objectId', 'predictions']
result = test[result_columns].sort_values(by=['instanceId_userId', 'predictions'], ascending=False)
result.head(10)

,instanceId_userId,instanceId_objectId,predictions
843591,15717369,10764030,0.006126
843590,15717369,30881127,0.005922
843589,15717369,30954311,0.005844
1181440,15717366,39141313,0.082090
1181441,15717366,39141920,0.075768
1181442,15717366,23761819,0.057999
1181437,15717360,38335200,0.186464
1181430,15717360,37863314,0.173221
1181435,15717360,36335739,0.152211
1181434,15717360,35706303,0.143227


In [10]:
submit = result.groupby("instanceId_userId")['instanceId_objectId'].apply(list)
submit.head(10)

instanceId_userId
39               [35321415, 18936317, 38436621, 38345738]
78                           [14594128, 38727626, 549335]
135     [17551814, 17738426, 18057011, 34080312, 38102...
180                                  [35804715, 12537921]
231                [30371401, 34978207, 975302, 30978017]
279                                   [12179433, 9471943]
681                        [20314212, 35151632, 35119221]
873                                  [18419723, 15483310]
1071    [25539638, 23720423, 36006622, 25050009, 35834...
1113                       [35894718, 29841332, 15781817]
Name: instanceId_objectId, dtype: object

Вот и все, сохраняем сабмит (он будет доступен во вкладке `Output` после работы кернела) и бегом проверять на [mlbootcamp](https://mlbootcamp.ru/round/17/tasks/)!

In [11]:
submit.to_csv('collab_submit.csv.gz', header=False, compression='gzip')